# Langchain Expression Language (LCEL)

In [ ]:
import json
from utils.helpers import get_secret
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import CohereEmbeddings
from langchain.schema.runnable import RunnableMap

In [ ]:
COHERE_KEY = get_secret("llm/api_keys/cohere")
PERSIST_DIR = "../docs/chroma/"

In [ ]:

embeddings = CohereEmbeddings(model="embed-english-light-v3.0", cohere_api_key=COHERE_KEY["cohere_api_key"])

## Simple Chain

In [ ]:
from langchain.chat_models import BedrockChat

In [ ]:
model = BedrockChat(model_id="anthropic.claude-v2", model_kwargs={"temperature": 0.4})

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
# model = ChatOpenAI()
model = model
output_parser = StrOutputParser()

In [ ]:
chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "anything"})

## Complex Chains

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Arko worked at Target", "bears like to eat honey"],
    # embedding=OpenAIEmbeddings()
    embedding=embeddings
)
retriever = vectorstore.as_retriever()

In [ ]:
retriever.get_relevant_documents("where did Arko work?")

In [ ]:
retriever.get_relevant_documents("what do bears like to eat")

In [ ]:
retriever.get_relevant_documents("is Arko a soft target?")


In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [ ]:
chain.invoke({"question": "where did arko work?"})

In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [ ]:
inputs.invoke({"question": "where did arko work?"})